# Model soup

In [1]:
import warnings 
warnings.filterwarnings('ignore')

import torch
import pandas as pd

#!pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch import ToTensorV2

from dataloader import CustomDataLoader
from SMP_dataset import test

dataset_path  = '../../data'
test_path = dataset_path + '/test.json'

batch_size = 8


test_transform = A.Compose([
                           ToTensorV2()
                           ])

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

pytorch version: 1.13.1+cu117
GPU 사용 가능 여부: True


In [2]:
# 기본코드
import inspect
import os
import sys
import time
import numpy as np

import torch
import segmentation_models_pytorch as smp
from models import build
build.register_encoder()
from mmcv import Config
# from mmseg.datasets import build_dataloader, build_dataset
# from mmseg.models import build_segmentor
# from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint, load_state_dict
from mmcv.parallel import MMDataParallel

def uniform_soup(model, checkpoint_paths ,device = "cpu", by_name = False):
    try:
        import torch
    except:
        print("If you want to use 'Model Soup for Torch', please install 'torch'")
        return model
    
    dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)
    data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size,
                                          num_workers=4,
                                          collate_fn=collate_fn)
    
    
    model = model.to(device)
    model_dict = model.state_dict()
    soups = {key:[] for key in model_dict}
    checkpoint = {}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        # checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        # state_dict = checkpoint.state_dict()
        # model.load_state_dict(state_dict)
        # weight_dict = checkpoint['state_dict']
        weight_dict = checkpoint.state_dict()
        for k, v in weight_dict.items():
            soups[k].append(v)
    if 0 < len(soups):
        soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
        model_dict.update(soups)
        model.load_state_dict(model_dict)
    
    return model, checkpoint

0. 모델 & checkpoint 가져오기

In [3]:
################ model cfg path 적기 ################
# cfg= Config.fromfile('/opt/ml/mmsegmentation/configs/_teajun_/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes.py')
################ model cfg path 적기 ################
model = smp.PAN(
			encoder_name="swin_encoder",
			encoder_weights="imagenet",
            encoder_output_stride=32,
			in_channels=3,
			classes=11
)

################ soup할 checkpoint path 적기 ################
checkpoint_paths = [
    '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/code/saved/SMP_PAN_SwinL_StepLR_AUG2_best_model(pretrained).pt',
    '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/code/saved/SMP_PAN_SwinL_StepLR_AUG2_CDBloss_best_model(pretrained).pt',
    '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/code/saved/SMP_PAN_SwinL_StepLR_AUG2_DFloss_best_model(pretrained).pt',
]
################ soup할 checkpoint path 적기 ################
device = "cpu"

1. uniform soup

In [4]:
################ save dir path 적기 ################
save_dir_path = '/opt/ml/input/level2_semanticsegmentation_cv-level2-cv-09/code/saved/'
name = 'AUG2LossSoup' # soup 이름 적기
################ save dir path 적기 ################

print("\n[Uniform Soup]")
uniform_model, checkpoint = uniform_soup(model, checkpoint_paths, device = device)
uniform_dict = checkpoint
# uniform_dict['state_dict'] = uniform_model.state_dict()


# torch.save(uniform_dict, save_dir_path+f'uniform_model_soup_{name}.pth')
file_name = f'uniform_model_soup_{name}.pt'
output_path = os.path.join(save_dir_path, file_name)
torch.save(uniform_model, output_path)
print("SAVED")


[Uniform Soup]
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
SAVED
